In [ ]:
%%capture
!pip3 install ../DISS/Tokenization
!pip3 install ../tkseem/
!pip3 install .


In [ ]:
!pip install ipywidgets==7.7.1

In [ ]:
!rm -rf ckpts/WordTokenizer

In [1]:
import nmatheg as nm
strategy = nm.TrainStrategy(
    datasets = 'ajgt_twitter_ar', 
    models   = 'birnn', 
    tokenizers = 'WordTokenizer',
    vocab_sizes = '500',
    runs = 3,
    lr = 1e-4,
    epochs = 20,
    batch_size = 64,
    max_tokens = 128, 
)
output = strategy.start()

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
!rm -rf /home/zaid/.cache/huggingface/datasets/ajgt_twitter_ar/

In [ ]:
!rm -rf ckpts/WordTokenizer

In [ ]:
import nmatheg as nm
strategy = nm.TrainStrategy('config.ini')
strategy.start()

In [ ]:
config = '''
[dataset]
dataset_name = ajgt_twitter_ar
task = classification 

[preprocessing]
segment = False
remove_special_chars = False
remove_english = False
normalize = False
remove_diacritics = False
excluded_chars = []
remove_tatweel = False
remove_html_elements = False
remove_links = False 
remove_twitter_meta = False
remove_long_words = False
remove_repeated_chars = False

[tokenization]
tokenizer_name = WordTokenizer
vocab_size = 10000
max_tokens = 128

[model]
model_name = aubmindlab/bert-base-arabertv01

[log]
print_every = 100

[train]
save_dir = .
epochs = 3
batch_size = 8 
''' 

with open('config.ini', 'w') as f:
    f.write(config)

In [ ]:
import nmatheg as nm
strategy = nm.TrainStrategy('config.ini')
strategy.start()

In [ ]:
!pip install .

In [ ]:
!pip install /home/zaid/DISS/Tokenization/.

In [ ]:
from bpe_surgery import bpe
tokenizer = bpe(vocab_size = 500)

In [ ]:
tokenizer.train(file_path = 'data.txt')

In [ ]:
tokenizer.tokenize('-')

In [ ]:
from datasets import load_dataset
examples = load_dataset("squad")
examples = examples["train"].train_test_split(test_size = 0.01)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
# https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/run_qa_no_trainer.py
import re
import copy 

def overflow_to_sample_mapping(tokens, offsets, idx, max_len = 384, doc_stride = 256):
    fixed_tokens = []
    fixed_offsets = []
    sep_index = tokens.index(-100)
    question = tokens[:sep_index]
    context = tokens[sep_index+1:]
    q_offsets = offsets[:sep_index]
    c_offsets = offsets[sep_index+1:]
    q_len = len(question)
    c_len = len(context)
    st_idx = 0 
    samplings = []
    sequences = []

    while True:
        ed_idx = st_idx+max_len-q_len-1
        if ed_idx > max_len:
            break
        fixed_tokens.append((question+[-100]+context[st_idx:])[:max_len])
        fixed_offsets.append(q_offsets+[(0,0)]+[(s, e) for (s,e) in c_offsets[st_idx:ed_idx]])
        c_len = len(context)
        st_idx += doc_stride
        samplings.append(idx)
        sequences.append([0]*q_len+[None]+[1]*c_len)
        
        
    for i in range(len(fixed_tokens)):
        assert len(fixed_tokens[i]) == len(fixed_offsets[i])  
    return fixed_tokens, fixed_offsets, samplings, sequences

def prepare_features(examples, tokenizer, data_config, model_type = 'bert', max_len = 128):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    if 'bert' in model_type:
        tokenized_examples = tokenizer(
            examples["question"],
            examples["context"],
            truncation="only_second",
            max_length=384,
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )
        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        print(len(tokenized_examples["overflow_to_sample_mapping"]))
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
        # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        offset_mapping = tokenized_examples["offset_mapping"]
        # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
        # corresponding example_id and we will store the offset mappings.
        # Let's label those examples!
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []

        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)

            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = examples["answers"][sample_index]
            # If no answers are given, set the cls_index as answer.
            if len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Start/end character index of the answer in the text.
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != 1:
                    token_start_index += 1

                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != 1:
                    token_end_index -= 1

                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions"].append(cls_index)
                    tokenized_examples["end_positions"].append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)

        tokenized_examples["example_id"] = []

        for i in range(len(tokenized_examples["input_ids"])):
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            tokenized_examples["example_id"].append(examples["id"][sample_index])

            # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
            # position is part of the context or not.
            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]
        
        return tokenized_examples
        
    else:
        question_col, context_col = data_config['text'].split(",")
        tokenized_examples = copy.deepcopy(examples)
        input_ids = []
        offset_mapping = []
        sequence_ids = []
        sample_mapping = []

        for i, (question, context) in enumerate(zip(examples[question_col], examples[context_col])):
            offsets = []
            tokens = []
            sequences = []
            question_context = question + " <sp> "+context
            st = 0
            for word in question_context.split(" "):
                if word == "<sp>":
                    offsets.append((0, 0))
                    tokens.append(-100)
                    st = 0 
                else:    
                    token_ids = tokenizer(word)["input_ids"][1:-1]
                    token_strs = tokenizer.tokenize(word)
                    assert len(token_ids) == len(token_strs)
                    for j, token_id in enumerate(token_ids):
                        token_str = token_strs[j].replace("##", "")
                        tokens.append(token_id)
                        offsets.append((st, st+len(token_str)))
                        st += len(token_str)
                    st += 1 # for space
            tokens, offsets, samplings, sequences = overflow_to_sample_mapping(tokens, offsets, i, max_len = max_len)
            sample_mapping+=samplings
            input_ids += tokens
            offset_mapping+= offsets
            sequence_ids += sequences
        
        
        tokenized_examples = tokenized_examples.add_column("input_ids", input_ids)
        tokenized_examples = tokenized_examples.add_column("sequence_ids", sequence_ids)
        tokenized_examples = tokenized_examples.add_column("sample_mapping", sample_mapping)
        start_positions = []
        end_positions = []

        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            # cls_index = input_ids.index(tokenizer.cls_token_id)
            cls_index = 0 

            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_id = tokenized_examples['sequence_ids'][i]

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = examples["answers"][sample_index]
            # If no answers are given, set the cls_index as answer.
            if len(answers["answer_start"]) == 0:
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                # Start/end character index of the answer in the text.
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_id[token_start_index] != 1:
                    token_start_index += 1

                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_id[token_end_index] != 1:
                    token_end_index -= 1

                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    start_positions.append(cls_index)
                    end_positions.append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    start_positions.append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    end_positions.append(token_end_index + 1)

        example_id = []
        fixed_offset_mapping = []
        for i in range(len(tokenized_examples["input_ids"])):
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_id = sequence_ids[i]
            context_index = 1

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            example_id.append(examples["id"][sample_index])

            # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
            # position is part of the context or not.
            try:
                fixed_offset_mapping.append([
                    (o if sequence_id[k] == context_index else None)
                    for k, o in enumerate(offset_mapping[i])
                ])
            except:
                print(sequence_ids)
                print(offset_mapping)
                raise('error')
        tokenized_examples = tokenized_examples.add_column("offset_mapping", fixed_offset_mapping)
        tokenized_examples = tokenized_examples.add_column("start_positions", start_positions)
        tokenized_examples = tokenized_examples.add_column("end_positions", end_positions)     
        return tokenized_examples

In [ ]:
prepare_features(examples['test'], tokenizer, {"text":"question,context"}, model_type = 'birnn', max_len = 128)

In [ ]:
len(examples['test'])